In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to hear from you. How can I assist you today?


In [5]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
website = Website("Your Website Link Goes Here")
print(website.title)
print(website.text)

vCard - Personal Portfolio
Anmol Mittal
Software Engineer
Show Contacts
Email
98anmolm@gmail.com
LinkedIn
My Profile
Birthday
1998
Location
Massachusetts, USA
About
Resume
Contact
About me
Software Engineer @ MathWorks | USC CS Alum | Full-Stack Web Development | Gen AI | Mobile Development
At MathWorks, I’m driven by the challenge of turning complex technical requirements into elegant and efficient code. I have a proven track record of leading projects that enhance security, optimize content management, and streamline operations, all while collaborating closely with cross-functional teams to ensure the highest quality outcomes.
I’m well-versed in Java, Python, C++, and JavaScript, constantly honing my skills and staying abreast of the latest industry trends. My experience across various industries has equipped me with the versatility to adapt quickly and deliver under pressure.
What i'm doing
Generative AI Solutions
Develop innovative AI-driven solutions to solve complex problems acro

In [7]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(website))

You are looking at a website titled vCard - Personal Portfolio
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Anmol Mittal
Software Engineer
Show Contacts
Email
98anmolm@gmail.com
LinkedIn
My Profile
Birthday
1998
Location
Massachusetts, USA
About
Resume
Contact
About me
Software Engineer @ MathWorks | USC CS Alum | Full-Stack Web Development | Gen AI | Mobile Development
At MathWorks, I’m driven by the challenge of turning complex technical requirements into elegant and efficient code. I have a proven track record of leading projects that enhance security, optimize content management, and streamline operations, all while collaborating closely with cross-functional teams to ensure the highest quality outcomes.
I’m well-versed in Java, Python, C++, and JavaScript, constantly honing my skills and staying abreast of the latest industry trends. My experience across vari

In [10]:
# The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

# [
#     {"role": "system", "content": "system message goes here"},
#     {"role": "user", "content": "user message goes here"}
# ]
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [11]:
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, we’re starting off with a real brain teaser, huh? Alright, drumroll, please... the answer is 4. Shocking, I know!


In [12]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [13]:
messages_for(website)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled vCard - Personal Portfolio\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nAnmol Mittal\nSoftware Engineer\nShow Contacts\nEmail\n98anmolm@gmail.com\nLinkedIn\nMy Profile\nBirthday\n1998\nLocation\nMassachusetts, USA\nAbout\nResume\nContact\nAbout me\nSoftware Engineer @ MathWorks | USC CS Alum | Full-Stack Web Development | Gen AI | Mobile Development\nAt MathWorks, I’m driven by the challenge of turning complex technical requirements into elegant and efficient code. I have a proven track record of leading projects that enhance security, optimize content management, and streamline operations, all while 

In [14]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("Your Website Link Goes Here")

"# vCard - Personal Portfolio\n\n**Anmol Mittal**  \n*Software Engineer*\n\n### Overview\nAnmol Mittal is a skilled Software Engineer based in Massachusetts, USA, with a degree in Computer Science from the University of Southern California. His expertise encompasses full-stack web development, generative AI, and mobile app development.\n\n### About Me\nAnmol works at MathWorks, focusing on transforming complex technical requirements into efficient code. He has hands-on experience in various programming languages, including Java, Python, C++, and JavaScript. His professional journey has equipped him with the ability to adapt quickly across industries.\n\n### Current Work\n- **Generative AI Solutions:** Developing AI-driven solutions to address complex challenges.\n- **Full-Stack Web Development:** Crafting comprehensive web solutions ensuring optimal user experiences.\n- **Mobile App Development:** Creating high-performance applications for iOS and Android.\n- **Photography:** Producing

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("Your Website Link Goes Here")

# vCard - Personal Portfolio Summary

**Name:** Anmol Mittal  
**Title:** Software Engineer  
**Location:** Massachusetts, USA  
**Email:** 98anmolm@gmail.com  
**LinkedIn:** [My Profile](#)  

## About Me
- Software Engineer at MathWorks with a focus on full-stack web development, generative AI, and mobile development.
- USC CS Alum with expertise in Java, Python, C++, and JavaScript.
- Known for turning complex requirements into efficient code, leading projects to enhance security, and optimizing operations through collaboration with teams.

## Current Focus
- **Generative AI Solutions:** Developing AI-driven solutions across various industries.
- **Full-Stack Web Development:** Creating seamless web solutions from user interface to backend.
- **Mobile App Development:** Building high-performance mobile apps for iOS and Android.

## Skills
- Front-End Web Development: 80%
- Back-End Web Development: 90%
- Generative AI: 70%
- Data Structures and Algorithms: 90%

## Education
- **University of Southern California:** 2021 — 2023
- **Guru Gobind Singh Indraprastha University:** 2016-2020

## Experience
- **Software Engineer, MathWorks:** June 2023 — Present
- **Software Engineer Intern, MathWorks:** May 2022 — August 2022
- **Software Developer, AppDesk Services:** September 2021 — July 2022

## Testimonials
Anmol has received praise from colleagues for his quick adaptability, eagerness to learn, and significant contributions to projects. 

Overall, Anmol Mittal's portfolio showcases a skilled software engineer with a strong foundation in development and a commitment to innovation.